# Pratical test 3: Artificial Inteligence

# Preliminary notes

This is a formal evaluation. It accounts for 30% of your final score in the practical component of Artificial Inteligence module. Please observe the following rules during the entire evaluation:

1. You cannot speak to other students once that evaluation starts.
2. There will be 15 minutes at the beginning where you can ask questions, and 5 min in the middle, and another 5 min towards the end of the test. Wait for your professor to indicate when you can ask questions.
3. You can only consult the Python documentation site.
4. You cannot consult any previous notebooks you created or shared with you.
5. All the math formulae you need are included here.
6. You can only have your Jupyter Python and a broswer pointing to the Python documentation open. Any other open tabs or apps can invalidate your test.
7. Distributing the contents of this test is forbidden.

# 0 - Libraries

In [4]:
%pip install datasets
%pip install nltk

Note: you may need to restart the kernel to use updated packages.



In [5]:
#biblioteca LDA
import gensim
import gensim.corpora as corpora
from gensim.models.ldamodel import LdaModel

#biblioteca para importar o dataset que vamos usar
from datasets import load_dataset

#leitura das stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = list(set(stopwords.words('portuguese')))
print(stop)

#lista de pontuações
import string
pontuacoes = string.punctuation
print(pontuacoes)

# dicionário default
from collections import defaultdict

#para criar contadores
from collections import Counter

#para usar a função flat
import itertools

['elas', 'lhe', 'me', 'numa', 'sejam', 'houverem', 'foram', 'houveremos', 'houver', 'nossos', 'estava', 'às', 'mas', 'esteve', 'tenha', 'terá', 'estive', 'fomos', 'ele', 'meu', 'estejamos', 'estivéramos', 'houvéramos', 'entre', 'tem', 'pela', 'tivesse', 'nem', 'esses', 'eu', 'nosso', 'a', 'mais', 'tém', 'tenhamos', 'isto', 'meus', 'terei', 'esta', 'até', 'será', 'o', 'seria', 'houvemos', 'nossa', 'estavam', 'temos', 'houve', 'tinham', 'aquela', 'forem', 'fossem', 'muito', 'esse', 'houveríamos', 'na', 'e', 'teu', 'formos', 'fui', 'tínhamos', 'ao', 'estivessem', 'estamos', 'lhes', 'dele', 'seriam', 'à', 'aquele', 'terão', 'que', 'for', 'tenho', 'tu', 'depois', 'houveria', 'houverei', 'aquilo', 'tenham', 'estivemos', 'num', 'os', 'eram', 'está', 'aos', 'estão', 'minha', 'ou', 'hajamos', 'houvessem', 'estivera', 'ser', 'vocês', 'essa', 'deles', 'é', 'haver', 'estiveram', 'vos', 'tuas', 'seríamos', 'teremos', 'uma', 'houveram', 'pelas', 'te', 'hão', 'houvéssemos', 'estávamos', 'tivemos', 'p

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1 - LDA

load dataset

In [6]:
#load dataset
dataset = load_dataset("tclopess/sinopsys_movies_portuguese")

#convert the dataset to pandas using only the train portion
df_pandas = dataset['train'].to_pandas()

#obtaining all the sinopse texts and converting to list
docs_raw = df_pandas['sinopse'].to_list()[:1000]

a) Based on the data above, propose a preprocessing function for each document in the corpus. Explain your steps. (You can use the `.split()` and `replace()` functions)

Required steps: tokenization, removal of stopwords, removal of punctuation marks

In [7]:
docs_raw[:5]

['A história da primeira grande batalha da fase americana da Guerra do Vietnã e os soldados de ambos os lados que a travaram.',
 'Um negócio de drogas dá errado, deixando corpos. Um xerife atira em seu vice pelo dinheiro e um amnésico ferido atira no xerife e foge com $ 3.000.000. A DEA investiga, assim como o traficante pai do deputado morto.',
 'Quando o disc jockey Grant Mazzy se reporta à estação de rádio de seu porão na cidade canadense de Pontypool, ele pensa que é apenas mais um dia de trabalho. Mas quando ouve relatos de um vírus que transforma as pessoas em zumbis, Mazzy se barrica na cabine de rádio e tenta descobrir uma maneira de alertar seus ouvintes sobre o vírus e seu improvável modo de transmissão.',
 'Depois que o parceiro de um detetive de São Francisco é assassinado por terroristas da Irlanda do Norte, o policial se junta ao esquadrão antibomba para levá-los à justiça.',
 'Um adolescente prodígio tenso entra em uma faculdade de engenharia de ponta, mas se sente estra

In [8]:
def preprocessing(text):
    # tokens_no_stopwords -> percorre as sinopses e retorna a lista de palavras que não estão nas stop words
    tokens_no_stopwords = [x.lower() for x in text.split() if x.lower() not in stop]
    # tokens_no_punctuation -> percorre a lista de palavras anterior e retorna as palavras sem pontuação (utiliza o join porque o ciclo devolve uma lista de
    # carateres)
    tokens_no_punctuation = [''.join(char for char in token if char not in pontuacoes) for token in tokens_no_stopwords]
    # tokens -> percorre a lista de palavras anteriores e retorna as palavras sem serem vazias (isto deve-se ao facto de ele retirar os carateres de pontuação 
    # e deixar palavras vazias caso não haja mais nada nessa palavra)
    tokens = [token for token in tokens_no_punctuation if token]

    return tokens

docs_tokenized = [preprocessing(doc) for doc in docs_raw]

print(docs_tokenized[1])

['negócio', 'drogas', 'dá', 'errado', 'deixando', 'corpos', 'xerife', 'atira', 'vice', 'dinheiro', 'amnésico', 'ferido', 'atira', 'xerife', 'foge', '3000000', 'dea', 'investiga', 'assim', 'traficante', 'pai', 'deputado', 'morto']


b) Would you add a new stopword to the existing ones(nltk)? Which addition and why?

Tip: count the total occurrences of each token to make your decision.

In [9]:
dict_tokens = defaultdict(int)

for document in docs_tokenized:
    for token in document:
        dict_tokens[token]+=1

In [10]:
print(sorted(dict_tokens.items(), key=lambda item: item[1],reverse=True))

[('vida', 166), ('anos', 111), ('enquanto', 109), ('cidade', 98), ('jovem', 92), ('família', 87), ('dois', 82), ('pai', 79), ('casa', 78), ('mundo', 75), ('história', 72), ('sobre', 71), ('homem', 66), ('descobre', 62), ('nova', 60), ('filho', 59), ('deve', 58), ('contra', 57), ('onde', 56), ('guerra', 55), ('amor', 55), ('filme', 52), ('tudo', 49), ('amigos', 49), ('apenas', 47), ('começa', 46), ('após', 45), ('encontrar', 45), ('mãe', 44), ('todos', 43), ('agente', 42), ('durante', 41), ('terra', 40), ('busca', 40), ('novo', 40), ('tempo', 40), ('filha', 39), ('pode', 39), ('mulher', 38), ('esposa', 38), ('grupo', 38), ('vez', 38), ('morte', 38), ('três', 36), ('volta', 36), ('missão', 36), ('agora', 36), ('luta', 35), ('torna', 35), ('grande', 34), ('ajuda', 34), ('the', 34), ('assassinato', 33), ('amigo', 33), ('fica', 33), ('fazer', 33), ('antes', 32), ('local', 32), ('irmão', 32), ('caso', 32), ('policial', 31), ('série', 31), ('york', 31), ('salvar', 31), ('primeiro', 30), ('cam

In [11]:
# Eu adicionaria as palavras, por exemplo, 'enquanto','dois','sobre', porque estas têm função estritamente gramatical, ou seja, não nos ajudam na nossa
# análise de tópicos e ou temas

c) Using the pre-processed data from exercises a) and b), input the data into the code below and obtain the LDA outputs.

In [12]:
# Create Dictionary
id2word = corpora.Dictionary(docs_tokenized)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in docs_tokenized]

# Build LDA model
lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=15,
    passes=20,
    random_state=200
    )

d) Explain the outputs below from the model:

In [13]:
lda_model.print_topics(num_words=20)

[(0,
  '0.005*"vida" + 0.004*"homem" + 0.004*"danny" + 0.004*"the" + 0.003*"esposa" + 0.003*"anos" + 0.003*"enquanto" + 0.003*"amor" + 0.003*"assassinato" + 0.002*"todos" + 0.002*"filho" + 0.002*"amigos" + 0.002*"chefe" + 0.002*"família" + 0.002*"dois" + 0.002*"descobre" + 0.002*"caso" + 0.002*"nome" + 0.002*"2" + 0.002*"michael"'),
 (1,
  '0.007*"casa" + 0.005*"sobre" + 0.004*"amigo" + 0.004*"amigos" + 0.004*"vida" + 0.004*"história" + 0.003*"família" + 0.003*"filme" + 0.003*"melhor" + 0.003*"mal" + 0.003*"nova" + 0.003*"série" + 0.003*"filho" + 0.003*"descobre" + 0.003*"apenas" + 0.003*"esposa" + 0.002*"dois" + 0.002*"grupo" + 0.002*"enquanto" + 0.002*"mãe"'),
 (2,
  '0.005*"vida" + 0.005*"guerra" + 0.004*"pai" + 0.004*"the" + 0.003*"anos" + 0.003*"fred" + 0.003*"onde" + 0.003*"cidade" + 0.003*"torna" + 0.003*"busca" + 0.003*"família" + 0.003*"jovem" + 0.003*"apenas" + 0.003*"homens" + 0.003*"vez" + 0.002*"descobre" + 0.002*"filme" + 0.002*"novo" + 0.002*"sangue" + 0.002*"país"'),
 (

In [14]:
# Este output representa 15 tópicos calculados pelo LDA, cada tópico contém uma lista de termos com os seus correspondentes pesos. Palavras com maiores pesos
# possuem uma maior significância nesses tópicos, podendo assim caracterizar melhor o tópico, mas esta caracterização não é linear, visto que quer no tópico 1,
# quer no tópico 4, o termo mais significante é "vida", mas num dos tópicos existe uma maior significância do que no outro.

In [15]:
for i, doc in enumerate(corpus):
    doc_topics = lda_model.get_document_topics(doc)
    print(f"Document {i}: {doc_topics}")

Document 0: [(4, 0.92818254)]
Document 1: [(7, 0.90383875), (9, 0.060036436)]
Document 2: [(3, 0.03312526), (6, 0.8335768), (14, 0.11106512)]
Document 3: [(5, 0.9333085)]
Document 4: [(4, 0.9626559)]
Document 5: [(13, 0.95554185)]
Document 6: [(8, 0.9827113)]
Document 7: [(5, 0.9377584)]
Document 8: [(7, 0.9666562)]
Document 9: [(8, 0.9333133)]
Document 10: [(2, 0.16522032), (5, 0.8113478)]
Document 11: [(0, 0.9508622)]
Document 12: [(2, 0.9575604)]
Document 13: [(0, 0.9654217)]
Document 14: [(12, 0.5210963), (14, 0.46445113)]
Document 15: [(10, 0.9594118)]
Document 16: [(1, 0.93331295)]
Document 17: [(11, 0.95756066)]
Document 18: [(14, 0.94164294)]
Document 19: [(7, 0.9481308)]
Document 20: [(4, 0.9066322)]
Document 21: [(9, 0.9575647)]
Document 22: [(0, 0.92818743)]
Document 23: [(14, 0.98606503)]
Document 24: [(10, 0.86663604)]
Document 25: [(6, 0.97970486)]
Document 26: [(2, 0.9481279)]
Document 27: [(3, 0.9481359)]
Document 28: [(14, 0.9450793)]
Document 29: [(2, 0.61608845), (9,

In [16]:
# Neste segundo output, podemos observar a lista de todos os documentos, e os tópicos pesentes nele e os pesos dos mesmos, ou seja, por exemplo no tópico 10
# temos 2 tópicos presentes, o 2 e o 5, sendo que o 5 possui uma maior probabilidade (0.81), podemos concluir que este documento é mais provável de ter um
# tema muito relacionado com o tópico 5 e menos com o tópico 2. 

e) The second output presented in letter d describes the mix of topics in each document. Define a criterion and classify the documents that belong to topic 10. List them.

In [17]:
# Lista para armazenar os documentos com o tópico 10
list_docs_topic10 = []

# Percorrer como no segundo output da alinea d)
for i, doc in enumerate(corpus):
    doc_topics = lda_model.get_document_topics(doc)

    # Para cada documento, verificar se tem o tópico 10
    for topic,percent in doc_topics:
        if topic==10 and percent>.50:
            print(f"Document {i}: {doc_topics}")
            list_docs_topic10.append(i)

# Critério foi apenas aceitar documentos que tiverem o documento 10 com percentagem maior que 50, para apenas filtrar os documentos com uma significância
# mais elevada deste tópico

Document 15: [(10, 0.9594118)]
Document 24: [(10, 0.86663604)]
Document 119: [(10, 0.937763)]
Document 140: [(10, 0.9066349)]
Document 149: [(10, 0.93331814)]
Document 188: [(10, 0.96110207)]
Document 232: [(10, 0.95332146)]
Document 263: [(10, 0.9626576)]
Document 289: [(10, 0.94813615)]
Document 290: [(10, 0.94165194)]
Document 327: [(10, 0.977772)]
Document 329: [(10, 0.9611007)]
Document 339: [(10, 0.9827115)]
Document 357: [(10, 0.93331414)]
Document 358: [(10, 0.95086634)]
Document 375: [(10, 0.9555434)]
Document 377: [(10, 0.9508637)]
Document 394: [(10, 0.96109986)]
Document 415: [(10, 0.97666067)]
Document 436: [(10, 0.9450874)]
Document 445: [(10, 0.97543156)]
Document 448: [(10, 0.94508296)]
Document 455: [(10, 0.91512483)]
Document 461: [(10, 0.96888083)]
Document 491: [(10, 0.93331164)]
Document 497: [(10, 0.9747681)]
Document 508: [(10, 0.92818385)]
Document 533: [(10, 0.9640947)]
Document 541: [(10, 0.9626589)]
Document 550: [(9, 0.26260823), (10, 0.71189153)]
Document 5